In [1]:
import numpy as np
import open3d as o3d

## Visualize point cloud

In [2]:
print("Load a ply point cloud, print it, and render it")
# ply_point_cloud = o3d.data.PLYPointCloud()
pcd = o3d.io.read_point_cloud("/Users/ignaciopastorebenaim/Documents/MGRCV/TPs/ASLAM/Lab1/fragment.ply")
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

Load a ply point cloud, print it, and render it
PointCloud with 196133 points.
[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]


In [ ]:
# Function to display registration results
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    source_temp_downsampled = source_temp.voxel_down_sample(voxel_size=0.01) # downsampling to avoid running out of memory in Colab
    target_temp_downsampled = target_temp.voxel_down_sample(voxel_size=0.01) # downsampling to avoid running out of memory in Colab
    pcd = source_temp_downsampled + target_temp_downsampled
    visualize_point_cloud(pcd)

In [ ]:
def visualize_reference_frame(fig, tmatrix):

  R = tmatrix[0:3,0:3]
  tx = tmatrix[0,3]
  ty = tmatrix[1,3]
  tz = tmatrix[2,3]
  axes_size = 0.05

  x = np.asarray([tx, tx+axes_size*(R[0,0]), tx+axes_size*(R[0,1]), tx+axes_size*(R[0,2])])
  y = np.asarray([ty, ty+axes_size*(R[1,0]), ty+axes_size*(R[1,1]), ty+axes_size*(R[1,2])])
  z = np.asarray([tz, tz+axes_size*(R[2,0]), tz+axes_size*(R[2,1]), tz+axes_size*(R[2,2])])

  pairs = [(0,1), (0,2), (0,3)]

  x_lines = list()
  y_lines = list()
  z_lines = list()

  for p in pairs:
    for i in range(2):
      x_lines.append(x[p[i]])
      y_lines.append(y[p[i]])
      z_lines.append(z[p[i]])
    x_lines.append(None)
    y_lines.append(None)
    z_lines.append(None)

  ## set the mode to lines to plot only the lines and not the balls/markers
  trace2 = go.Scatter3d(
      x=x_lines[0:2],
      y=y_lines[0:2],
      z=z_lines[0:2],
      mode='lines',
      line = dict(width = 4, color = 'rgb(255, 0, 0)')
  )

  trace3 = go.Scatter3d(
      x=x_lines[3:5],
      y=y_lines[3:5],
      z=z_lines[3:5],
      mode='lines',
      line = dict(width = 4, color = 'rgb(0, 255, 0)')
  )

  trace4 = go.Scatter3d(
      x=x_lines[6:8],
      y=y_lines[6:8],
      z=z_lines[6:8],
      mode='lines',
      line = dict(width = 4, color = 'rgb(0, 0, 255)')
  )

  #fig = go.Figure(data=[trace2, trace3, trace4])
  fig.add_trace(trace2)
  fig.add_trace(trace3)
  fig.add_trace(trace4)

  arrow_tip_ratio = 0.3
  arrow_starting_ratio = 0.98

  ## the cone will point in the direction of vector field u, v, w
  ## so we take this to be the difference between each pair

  colors = ['rgb(255,0,0)', 'rgb(0,255,0)', 'rgb(0,0,255)']
  for p in pairs:
      fig.add_trace(go.Cone(
          x=[x[p[0]] + arrow_starting_ratio*(x[p[1]] - x[p[0]])],
          y=[y[p[0]] + arrow_starting_ratio*(y[p[1]] - y[p[0]])],
          z=[z[p[0]] + arrow_starting_ratio*(z[p[1]] - z[p[0]])],
          u=[arrow_tip_ratio*(x[p[1]] - x[p[0]])],
          v=[arrow_tip_ratio*(y[p[1]] - y[p[0]])],
          w=[arrow_tip_ratio*(z[p[1]] - z[p[0]])],
          showscale=False,
          colorscale=[[0, colors[p[1]-1]], [1, colors[p[1]-1]]]
          ))
  fig.update_layout(showlegend=False)

In [ ]:
# Read a point cloud from a file and visualize it

# fragment.ply can be downloaded from Moodle or from https://github.com/HuangCongQing/Point-Clouds-Visualization/blob/master/2open3D/data/fragment.ply
cloud = o3d.io.read_point_cloud('/content/drive/MyDrive/uni/teaching/ASLAM_MURGCV/l1/fragment.ply')
dcloud = cloud.voxel_down_sample(voxel_size=0.1) # downsampling to avoid running out of memory in Colab
visualize_point_cloud(dcloud)

[Open3D WARNING] Read PLY failed: unable to open file: /content/drive/MyDrive/uni/teaching/ASLAM_MURGCV/l1/fragment.ply


In [ ]:
# Functions to read files containing the ground truth camera poses

class CameraPose:
    def __init__(self, meta, mat):
        self.metadata = meta
        self.pose = mat
    def __str__(self):
        return 'Metadata : ' + ' '.join(map(str, self.metadata)) + '\n' + \
            "Pose : " + "\n" + np.array_str(self.pose)

def read_trajectory(filename):
    traj = []
    with open(filename, 'r') as f:
        metastr = f.readline();
        while metastr:
            metadata = map(int, metastr.split())
            mat = np.zeros(shape = (4, 4))
            for i in range(4):
                matstr = f.readline();
                mat[i, :] = np.fromstring(matstr, dtype = float, sep=' \t')
            traj.append(CameraPose(metadata, mat))
            metastr = f.readline()
    return traj

In [ ]:
# Example reading poses from file
gtposes = read_trajectory('/content/drive/MyDrive/uni/teaching/ASLAM_MURGCV/l1/livingroom1-traj.txt')
print(gtposes[33].pose)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/uni/teaching/ASLAM_MURGCV/l1/livingroom1-traj.txt'

In [ ]:
# Example displaying a reference frame
fig = go.Figure()
visualize_reference_frame(fig, gtposes[33].pose)
fig.update_layout(showlegend=False)

In [ ]:
# Example converting a rotation matrix to Euler angles
from scipy.spatial.transform import Rotation as R
r = R.from_matrix(np.matmul(np.linalg.inv(gtposes[33].pose[:3,:3]),gtposes[34].pose[:3,:3]))
print(r.as_euler('zxy', degrees=True))

In [ ]:
# Example for reading the image files in a folder
import os
import matplotlib.pyplot as plt

my_path = '/content/drive/MyDrive/uni/teaching/ASLAM_MURGCV/l1/livingroom1-color/'
rgdfiles = sorted(os.listdir(my_path))
print(rgdfiles)
for i in range(10): #range(len(rgdfiles)):
  print(rgdfiles[i])
  color_raw = o3d.io.read_image('/content/drive/MyDrive/uni/teaching/ASLAM_MURGCV/l1/livingroom1-color/%(number)05d.jpg'%{"number": i})
  print(color_raw)
  plt.imshow(color_raw)
  plt.show()